In [5]:
pip install yfinance pandas scikit-learn ta matplotlib


In [6]:
import yfinance as yf
import pandas as pd
import time
from ta.momentum import RSIIndicator
from ta.trend import MACD
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

# Fetch data with retry
def fetch_data(ticker='AAPL', period='6mo', max_retries=5):
    for i in range(max_retries):
        try:
            df = yf.download(ticker, period=period, progress=False)
            if df.empty:
                raise ValueError("Data is empty.")
            return df
        except Exception as e:
            print(f"Attempt {i+1} failed: {e}")
            time.sleep((i+1) * 5)
    raise RuntimeError("Failed to fetch data after retries.")

# Feature engineering
def add_indicators(df):
    df['RSI'] = RSIIndicator(df['Close']).rsi()
    macd = MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['Signal'] = macd.macd_signal()
    df['Price_Change'] = df['Close'].pct_change()
    df.dropna(inplace=True)
    return df

# Label generation
def generate_labels(df, threshold=0.003):
    df['Target'] = df['Price_Change'].shift(-1).apply(lambda x: 1 if x > threshold else 0)
    df.dropna(inplace=True)
    return df

# Model training
def train_model(df):
    features = ['RSI', 'MACD', 'Signal']
    X = df[features]
    y = df['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Report:\n", classification_report(y_test, y_pred))

    return model

# Backtesting
def simulate_trading(df, model):
    features = ['RSI', 'MACD', 'Signal']
    df['Prediction'] = model.predict(df[features])
    df['Strategy_Return'] = df['Prediction'].shift(1) * df['Price_Change']
    df['Market_Cumulative'] = (1 + df['Price_Change']).cumprod()
    df['Strategy_Cumulative'] = (1 + df['Strategy_Return']).cumprod()

    df[['Market_Cumulative', 'Strategy_Cumulative']].plot(title="Strategy vs Market Returns")
    plt.show()
    return df

# Run the system
def run_trading_bot(ticker='AAPL'):
    df = fetch_data(ticker)
    df = add_indicators(df)
    df = generate_labels(df)
    model = train_model(df)
    result = simulate_trading(df, model)
    return result

# Execute
if __name__ == "__main__":
    run_trading_bot("AAPL")


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Attempt 1 failed: Data is empty.


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Attempt 2 failed: Data is empty.


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


...

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Attempt 4 failed: Data is empty.


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Attempt 5 failed: Data is empty.


RuntimeError: Failed to fetch data after retries.